In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json
from ensemble_boxes_wbf import *
import cv2

In [25]:
test_fnames = os.listdir('data/medical/img/test')
test_path = 'data/medical/img/test'
print(test_fnames[:5])

['drp.en_ko.in_house.deepnatural_002804.jpg', 'drp.en_ko.in_house.deepnatural_003610.jpg', 'drp.en_ko.in_house.deepnatural_002535.jpg', 'drp.en_ko.in_house.deepnatural_002576.jpg', 'drp.en_ko.in_house.deepnatural_003511.jpg']


In [26]:
#========== submission.csv 파일 경로 지정 ===========#
submission_files = ['predictions/T6132_develop_mean_std_output.csv',
                    'predictions/T6132_GaussNoise_output.csv']
#===================================================#

results = {}
iou_thr = 0.5            # TODO : iou threshold 설정
skip_box_thr = 0.0001    # TODO : skip iou threshold 설정 0.0001

In [27]:
for fname in tqdm(test_fnames):
    boxes_list = []
    scores_list = []
    labels_list = []

    img = cv2.imread(os.path.join(test_path, fname))
    height, width, c = img.shape

    for submission in submission_files:
        with open(submission, 'r') as f:
            data = json.load(f)
            images = data['images']
        words = images[fname]
        box_list = []


        for word_id in words["words"].keys():
            points = words['words'][word_id]['points']
            # left_top, right_top, right_bottom, left_bottom
            p1, p2, p3, p4 = points    
            xmin = min(p1[0], p2[0], p3[0], p4[0])
            ymin = min(p1[1], p2[1], p3[1], p4[1])
            xmax = max(p1[0], p2[0], p3[0], p4[0])
            ymax = max(p1[1], p2[1], p3[1], p4[1])
            # 좌상단 우하단
            box = [xmin/width, ymin/height, xmax/width, ymax/height]
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append([1.0]*len(words["words"].items()))
        labels_list.append([1]*len(words["words"].items()))

    if len(boxes_list):
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(box_list, scores_list, labels_list, iou_thr=iou_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        
        prediction_words = []
        points_list = []
        for idx, (box, score, label) in enumerate(zip(boxes, scores, labels)):
            xmin = box[0]*width
            ymin = box[1]*height
            xmax = box[2]*width
            ymax = box[3]*height
            
            p1 = [xmin, ymin]
            p2 = [xmin + width, ymin]
            p3 = [xmin + width, ymin + height]
            p4 = [xmin, ymin + height]
            points = [p1, p2, p3, p4]
            points_list.append(points)
        prediction_words = {idx: dict(points=point) for idx, point in enumerate(points_list)}
    results[fname] = dict(words=prediction_words)
results = {'images' :results}

output_path = 'predictions/ensemble'
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
output_fname = 'output.csv'
with open(os.path.join(output_path, output_fname), 'w') as f:
    json.dump(results, f, indent=4)

  1%|          | 1/100 [00:00<01:15,  1.32it/s]/data/ephemeral/home/level2-cv-datacentric-cv-13/ensemble_boxes_wbf.py:38: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
 18%|█▊        | 18/100 [00:11<00:54,  1.50it/s]/data/ephemeral/home/level2-cv-datacentric-cv-13/ensemble_boxes_wbf.py:59: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
 31%|███       | 31/100 [00:18<00:38,  1.78it/s]/data/ephemeral/home/level2-cv-datacentric-cv-13/ensemble_boxes_wbf.py:50: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')
 76%|███████▌  | 76/100 [00:47<00:16,  1.41it/s]/data/ephemeral/home/level2-cv-datacentric-cv-13/ensemble_boxes_wbf.py:47: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you 